In [1]:
import cv2
import numpy as np

In [4]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

# Shape Detection 

In [39]:
img = cv2.imread('shapes.png')
img_contours = img.copy()
img_g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_bl = cv2.GaussianBlur(img_g, (7, 7), 1)
img_c = cv2.Canny(img_bl, 50, 50)

img_b = np.zeros_like(img)

def getContours(img):
    contours, hirarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        print(area)
        if area > 500:
            cv2.drawContours(img_contours, cnt, -1, (255, 0, 0),3)
            pari = cv2.arcLength(cnt, True)
            #print(pari)
            approx = cv2.approxPolyDP(cnt, 0.02*pari, True)
            print(len(approx))
            objCor = len(approx)
            
            x, y, w, h = cv2.boundingRect(approx)
            
            cv2.rectangle(img_contours, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            if objCor == 3: objectType="Triangle"
            elif objCor == 4:
                absRatio = w/float(h)
                if absRatio >0.95 and absRatio <1.05: objectType="Square"
                else: objectType="Rectangle"
            elif objCor > 4: objectType="Circle"
            else: objectType="None"
                
            cv2.putText(img_contours, objectType, (x+(w//2)-10, y+(h//2)-10), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,0), 2)

getContours(img_c)

StackImg = stackImages(0.5, [[img, img_g, img_bl],
                            [img_c, img_contours, img_b]])

cv2.imshow('StackImg', StackImg)
cv2.waitKey(0)

3593.5
8
8619.0
4
10398.5
4
10236.0
8
1612.5
3
10402.0
4
6389.0
4
5629.0
3
3475.5
4
2270.5
4
3552.5
8
8674.0
4
2646.0
4
5690.5
3
10234.0
8
5712.0
3


-1